In [1]:
using NBInclude
@nbinclude("functions.ipynb")

All function imported


In [2]:
#setup
x_dim = 784
k = 20
epoch_to_load = 20 # Load the model 
tolerance = 1e-7; max_iter = 3000; out_toggle = 0
trials = 20
opt = Flux.Optimise.ADAM(.1);

_, _, decoder = load_model("trained_GNN/MNIST", epoch_to_load)
B = randn(784, 500)/sqrt(784)
z = randn(k) # ground code vector
F = dct(diagm(ones(x_dim)),2);

β_list, α_list = get_β_α(F, Flux.params(decoder)[3], B, 21);
m_list = 15:5:150
# setup
# experiments with MNIST dataset

Loading model...Done

15:5:150

In [ ]:
phase_matrix_recovery = zeros(trials, length(m_list), length(β_list))
phase_matrix_recon = zeros(trials, length(m_list), length(β_list))

Threads.@threads for i in 1:length(m_list)
    m = m_list[i]
    recovery_error_matrix = zeros(trials)
    recon_error_matrix = zeros(trials)
    for β in β_list
        recovery_error_list = []
        recon_error_list = []
        G = Chain(
            Dense(k, n1, relu, bias = false; init =(out,in) -> W1),
            Dense(n1, x_dim, identity, bias = false; init =(out,in) -> β*A + (1-β) * B)
        )    #setup the generative network 

        for trials in 1:trials
            F_sub = sample_fourier(m, x_dim)  # subsampling DCT for measurement matrix
            F_sub = F_sub * sqrt(x_dim)/sqrt(m) #normalization          
            y = F_sub * G(z) # measurement vector using subsampled DCT matrix
    
            z_est = randn(k) #initialization for algorithm
            z_est = estimated_code(opt, G, y, F_sub, z_est; max_iter, tolerance, out_toggle)  # run a optimizer to solve the least squares problem

            push!(recovery_error_list, relative_error(z, z_est))    
            push!(recon_error_list, relative_error(G(z), G(z_est)))
        end

        recovery_error_matrix =  hcat(recovery_error_matrix, recovery_error_list)
        recon_error_matrix = hcat(recon_error_matrix, recon_error_list)
    end
    phase_matrix_recovery[:, i, :] = recovery_error_matrix[:, 2:end]
    phase_matrix_recon[:, i, :] = recon_error_matrix[:, 2:end]
    println("Done with m = $m")
    println("")
end

In [19]:

β = 1
tolerance = 1e-7; max_iter = 4000; out_toggle = 0

m = 500
G = Chain(
    Dense(20, 500, relu, bias = false; init =(out,in) -> Flux.params(decoder)[1]),
    Dense(500, 500, relu, bias = false; init =(out,in) -> Flux.params(decoder)[2]),
    Dense(500, 784, identity, bias = false; init =(out,in) -> β * Flux.params(decoder)[3] + (1-β)*B),
)    #setup the generative network 

F_sub = sample_fourier(m, x_dim)  # subsampling DCT for measurement matrix
F_sub = F_sub * sqrt(x_dim)/sqrt(m) #normalization          
y = F_sub * G(z) # measurement vector using subsampled DCT matrix

z_est = randn(k) #initialization for algorithm
z_est = estimated_code(opt, G, y, F_sub, z_est; max_iter, tolerance, out_toggle)  # run a optimizer to solve the least squares problem

relative_error(z, z_est)

0.00034153750711776243

In [6]:
relative_error(z, z_est)

G

Chain(
  Dense(20 => 500, relu; bias=false),   # 10_000 parameters
  Dense(500 => 500, relu; bias=false),  # 250_000 parameters
  Dense(500 => 784; bias=false),        # 392_000 parameters
)                   # Total: 3 arrays, 652_000 parameters, 3.983 MiB.